以下是使用 Python 实现上述要求的步骤：

1. 数据预处理：
- 平滑滤波（Savitzky-Golay, S-G）：可以使用  scipy.signal.savgol_filter  函数进行 S-G 滤波。
- 标准正态变换（Standard Normal Variate, SNV）：可以使用  sklearn.preprocessing.StandardScaler  进行 SNV 变换。
- 主成分分析（Principal Component Analysis, PCA）：可以使用  sklearn.decomposition.PCA  进行 PCA 分析。

2. 划分训练集和测试集： 使用  train_test_split  函数从数据集中随机抽取 50 个样本作为训练集，剩余 10 个样本作为测试集。

3. 构建 BP-ANN 神经网络模型： 使用  Sequential  模型构建 BP-ANN 神经网络，添加适当的隐藏层和激活函数。

4. 训练模型并进行预测： 使用训练集对模型进行训练，然后对测试集进行预测，得到预测值。

5. 计算评估指标： 使用  r2_score 、 mean_squared_error  等函数计算 R2、RMSE 相对误差等指标。

6. 可视化预测值与真实值的偏差： 可以使用 matplotlib 库绘制图表展示预测值与真实值的偏差。

以下是一个示例代码：


In [ ]:
import numpy as np
from scipy.signal import savgol_filter
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

# 读取数据文件
data = np.load('spectra data.mat')

# 划分训练集和测试集
x = data['x']
y = data['y']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 数据预处理
# S-G 滤波
sg_x_train = savgol_filter(x_train, window_length=5, polyorder=2)
sg_x_test = savgol_filter(x_test, window_length=5, polyorder=2)

# SNV 变换
scaler = StandardScaler()
sg_x_train = scaler.fit_transform(sg_x_train)
sg_x_test = scaler.transform(sg_x_test)

# PCA 分析
pca = PCA(n_components=2)
sg_x_train_pca = pca.fit_transform(sg_x_train)
sg_x_test_pca = pca.transform(sg_x_test)

# 构建 BP-ANN 神经网络模型
model = MLPClassifier(hidden_layer_sizes=(10, 5), activation='relu')
model.fit(sg_x_train_pca, y_train)

# 进行预测
y_pred = model.predict(sg_x_test_pca)

# 计算评估指标
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
relative_error = np.mean(np.abs(y_test - y_pred) / y_test) * 100

# 可视化预测值与真实值的偏差
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('Prediction Error')
plt.show()